In [12]:
import cv2
import mediapipe as mp
import numpy as np

In [13]:
mpsolution = mp.solutions.holistic
drawing = mp.solutions.drawing_utils

In [14]:
def Superimpose(image, landmarks):
    drawing.draw_landmarks(image, landmarks.face_landmarks, mpsolution.FACEMESH_CONTOURS)
    drawing.draw_landmarks(image, landmarks.pose_landmarks, mpsolution.POSE_CONNECTIONS)
    drawing.draw_landmarks(image, landmarks.left_hand_landmarks, mpsolution.HAND_CONNECTIONS)
    drawing.draw_landmarks(image, landmarks.right_hand_landmarks, mpsolution.HAND_CONNECTIONS)

In [18]:
#consider improving
def Brightness(image, gamma):
    result = np.ones(image.shape, dtype = "uint8") * gamma
    result = cv2.add(image, result)

    return result
    

In [16]:
def detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return result

In [17]:
video = cv2.VideoCapture(0)

with mpsolution.Holistic(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as model:
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        #frame = fixGama(frame, 40)
        result = detection(frame, model)
        print(result)
        
        Superimpose(frame, result)
        
        cv2.imshow("OnlyFans!!", frame)

        if cv2.waitKey(33) & 0xFF == ord('q'):
            break
    
    video.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [7]:
result.face_landmarks

In [23]:
video = cv2.VideoCapture(0)

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = Brightness(frame, 50)
    cv2.imshow("OnlyFans!!", frame)

    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()